In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv("C:/Users/dell/Desktop/ml/pet detectoin/Dataset/train.csv")

In [3]:
data.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2,breed_category,pet_category
0,ANSL_69903,2016-07-10 00:00:00,2016-09-21 16:25:00,2.0,Brown Tabby,0.80,7.78,13,9,0.0,1
1,ANSL_66892,2013-11-21 00:00:00,2018-12-27 17:47:00,1.0,White,0.72,14.19,13,9,0.0,2
2,ANSL_69750,2014-09-28 00:00:00,2016-10-19 08:24:00,NaN,Brown,0.15,40.90,15,4,2.0,4
3,ANSL_71623,2016-12-31 00:00:00,2019-01-25 18:30:00,1.0,White,0.62,17.82,0,1,0.0,2
4,ANSL_57969,2017-09-28 00:00:00,2017-11-19 09:38:00,2.0,Black,0.50,11.06,18,4,0.0,1


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18834 entries, 0 to 18833
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pet_id          18834 non-null  object 
 1   issue_date      18834 non-null  object 
 2   listing_date    18834 non-null  object 
 3   condition       17357 non-null  float64
 4   color_type      18834 non-null  object 
 5   length(m)       18834 non-null  float64
 6   height(cm)      18834 non-null  float64
 7   X1              18834 non-null  int64  
 8   X2              18834 non-null  int64  
 9   breed_category  18834 non-null  float64
 10  pet_category    18834 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 1.6+ MB


In [5]:
data.shape

(18834, 11)

In [6]:
data.isna().sum()

pet_id               0
issue_date           0
listing_date         0
condition         1477
color_type           0
length(m)            0
height(cm)           0
X1                   0
X2                   0
breed_category       0
pet_category         0
dtype: int64

In [7]:
data.breed_category.value_counts()

0.0    9000
1.0    8357
2.0    1477
Name: breed_category, dtype: int64

In [8]:
print("-------------- Numeric Data ---------------")
for label , content in data.items():
    if pd.api.types.is_numeric_dtype(content):
        print(label)

-------------- Numeric Data ---------------
condition
length(m)
height(cm)
X1
X2
breed_category
pet_category


In [9]:
print("-------------- Numeric Data ---------------")
for label , content in data.items():
    if not pd.api.types.is_numeric_dtype(content):
        print(label)

-------------- Numeric Data ---------------
pet_id
issue_date
listing_date
color_type


In [10]:
data_temp = data.copy()

In [11]:
data_temp.drop(["pet_category"] ,axis = 1, inplace = True)
data_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18834 entries, 0 to 18833
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pet_id          18834 non-null  object 
 1   issue_date      18834 non-null  object 
 2   listing_date    18834 non-null  object 
 3   condition       17357 non-null  float64
 4   color_type      18834 non-null  object 
 5   length(m)       18834 non-null  float64
 6   height(cm)      18834 non-null  float64
 7   X1              18834 non-null  int64  
 8   X2              18834 non-null  int64  
 9   breed_category  18834 non-null  float64
dtypes: float64(4), int64(2), object(4)
memory usage: 1.4+ MB


In [12]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# DATA PREPROCESSING.
cat_features = ["pet_id" , "issue_date" , "listing_date" , "color_type"]
num_features = ["condition" , "length(m)" , "height(cm)" , "X1" , "X2"]

cat_transformer = Pipeline(steps=[
    ("imputer" , SimpleImputer(strategy="constant" , fill_value="missing" )),
    ("encoder" , OneHotEncoder(handle_unknown="ignore" ))
])

numerical_transformer = Pipeline(steps=[
    ("imputer" , SimpleImputer(strategy="constant" , fill_value= 2.0))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("cat" , cat_transformer ,cat_features),
        ("num" , numerical_transformer , num_features)
    ])
model1 = Pipeline(steps=[
    ("preprocess" , preprocessor),
    ("model" , RandomForestClassifier())
])

In [13]:
x = data_temp.drop("breed_category" , axis = 1)
y = data_temp["breed_category"]
xtrain , xtest , ytrain , ytest = train_test_split(x,y,test_size= 0.25)

In [15]:
%%time
model1.fit(xtrain , ytrain)

Wall time: 10.4 s


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['pet_id', 'issue_date',
                                                   'listing_date',
                                                   'color_type']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=2.0

In [18]:
breed_predict = model1.predict(xtest)

In [19]:
breed_predict = pd.Series(breed_predict)

In [20]:
breed_predict.head()

0    1.0
1    1.0
2    1.0
3    1.0
4    0.0
dtype: float64

In [21]:
ytest.head()

16291    1.0
9479     1.0
13111    1.0
11427    1.0
6296     1.0
Name: breed_category, dtype: float64

In [22]:
breed_predict.value_counts()

0.0    2812
1.0    1704
2.0     193
dtype: int64

In [23]:
ytest.value_counts()

0.0    2258
1.0    2088
2.0     363
Name: breed_category, dtype: int64

In [24]:
model1.score(xtest , ytest)

0.8630282437884901

In [26]:
%%time
from sklearn.model_selection import GridSearchCV
pipe_grid = {
    "preprocess__num__imputer__strategy":["mean" , "median"],
    "model__n_estimators":[10,15],
    "model__max_depth":[None , 2 ,4],
    "model__max_features":["auto"],
    "model__min_samples_split":[2,4,6],
    "model__max_leaf_nodes":[None , 3,5]
}
gs_model = GridSearchCV(model1 , pipe_grid , cv=10,verbose=2)
gs_model.fit(xtrain , ytrain)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits
[CV] model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None, model__min_samples_split=2, model__n_estimators=10, preprocess__num__imputer__strategy=mean 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None, model__min_samples_split=2, model__n_estimators=10, preprocess__num__imputer__strategy=mean, total=   1.0s
[CV] model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None, model__min_samples_split=2, model__n_estimators=10, preprocess__num__imputer__strategy=mean 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV]  model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None, model__min_samples_split=2, model__n_estimators=10, preprocess__num__imputer__strategy=mean, total=   0.8s
[CV] model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None, model__min_samples_split=2, model__n_estimators=10, preprocess__num__imputer__strategy=mean 
[CV]  model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None, model__min_samples_split=2, model__n_estimators=10, preprocess__num__imputer__strategy=mean, total=   0.9s
[CV] model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None, model__min_samples_split=2, model__n_estimators=10, preprocess__num__imputer__strategy=mean 
[CV]  model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None, model__min_samples_split=2, model__n_estimators=10, preprocess__num__imputer__strategy=mean, total=   0.9s
[CV] model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None

...

[CV]  model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=10, preprocess__num__imputer__strategy=median, total=   0.1s
[CV] model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=10, preprocess__num__imputer__strategy=median 
[CV]  model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=10, preprocess__num__imputer__strategy=median, total=   0.1s
[CV] model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=10, preprocess__num__imputer__strategy=median 
[CV]  model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=10, preprocess__num__imputer__strategy=median, total=   0.1s
[CV] model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split

[CV]  model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=15, preprocess__num__imputer__strategy=median, total=   0.1s
[CV] model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=15, preprocess__num__imputer__strategy=median 
[CV]  model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=15, preprocess__num__imputer__strategy=median, total=   0.1s
[CV] model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=15, preprocess__num__imputer__strategy=median 
[CV]  model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=15, preprocess__num__imputer__strategy=median, total=   0.1s


[Parallel(n_jobs=1)]: Done 1080 out of 1080 | elapsed:  3.3min finished


Wall time: 3min 22s


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='constant')),
                                                                                         ('encoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['pet_id',
                                                                          'issue_date',
                                                                          'listing_dat

In [27]:
gs_model.score(xtest , ytest)

0.8953068592057761

In [28]:
breed_predict2 = gs_model.predict(xtest)

In [29]:
breed_predict2 = pd.Series(breed_predict2)

In [30]:
breed_predict2.value_counts()

0.0    2689
1.0    1674
2.0     346
dtype: int64

In [31]:
ytest.value_counts()

0.0    2258
1.0    2088
2.0     363
Name: breed_category, dtype: int64

In [32]:
from joblib import dump , load
dump(gs_model , filename = "breed_classifer_grid_search.joblib")

['breed_classifer_grid_search.joblib']

In [57]:
data_temp1 = data.copy()

In [58]:
data_temp1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18834 entries, 0 to 18833
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   pet_id          18834 non-null  object 
 1   issue_date      18834 non-null  object 
 2   listing_date    18834 non-null  object 
 3   condition       17357 non-null  float64
 4   color_type      18834 non-null  object 
 5   length(m)       18834 non-null  float64
 6   height(cm)      18834 non-null  float64
 7   X1              18834 non-null  int64  
 8   X2              18834 non-null  int64  
 9   breed_category  18834 non-null  float64
 10  pet_category    18834 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 1.6+ MB


In [59]:
data_temp1.isna().sum()

pet_id               0
issue_date           0
listing_date         0
condition         1477
color_type           0
length(m)            0
height(cm)           0
X1                   0
X2                   0
breed_category       0
pet_category         0
dtype: int64

In [60]:
data_temp1.drop("breed_category" , axis = 1 , inplace= True)
data_temp1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18834 entries, 0 to 18833
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   pet_id        18834 non-null  object 
 1   issue_date    18834 non-null  object 
 2   listing_date  18834 non-null  object 
 3   condition     17357 non-null  float64
 4   color_type    18834 non-null  object 
 5   length(m)     18834 non-null  float64
 6   height(cm)    18834 non-null  float64
 7   X1            18834 non-null  int64  
 8   X2            18834 non-null  int64  
 9   pet_category  18834 non-null  int64  
dtypes: float64(3), int64(3), object(4)
memory usage: 1.4+ MB


In [51]:
# STARTING WITH PET CATEGORY
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# DATA PREPROCESSING.
cat_features = ["pet_id" , "issue_date" , "listing_date" , "color_type"]
num_features = ["condition" , "length(m)" , "height(cm)" , "X1" , "X2"]

cat_transformer = Pipeline(steps=[
    ("imputer" , SimpleImputer(strategy="constant" , fill_value="missing" )),
    ("encoder" , OneHotEncoder(handle_unknown="ignore" ))
])

numerical_transformer = Pipeline(steps=[
    ("imputer" , SimpleImputer(strategy="constant" , fill_value= 2.0))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("cat" , cat_transformer ,cat_features),
        ("num" , numerical_transformer , num_features)
    ])
model2 = Pipeline(steps=[
    ("preprocess" , preprocessor),
    ("model" , RandomForestClassifier())
])

In [52]:
x = data_temp1.drop("pet_category" , axis=1)
y = data_temp1["pet_category"]
xtrain2 , xtest2 , ytrain2 ,ytest2 = train_test_split(x,y,test_size=0.2)

In [53]:
%%time
model2.fit(xtrain2 , ytrain2)

Wall time: 12.1 s


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['pet_id', 'issue_date',
                                                   'listing_date',
                                                   'color_type']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=2.0

In [54]:
model2.score(xtest2 , ytest2)

0.8855853464295195

In [61]:
%%time
from sklearn.model_selection import GridSearchCV
pipe_grid2 = {
    "preprocess__num__imputer__strategy":["mean" , "median"],
    "model__n_estimators":[10 , 15],
    "model__max_depth":[None , 2 ,4],
    "model__max_features":["auto"],
    "model__min_samples_split":[2,4,6],
    "model__max_leaf_nodes":[None , 3,5]
}
gs_model_2 = GridSearchCV(model2 , pipe_grid2,cv=10,verbose=2)
gs_model_2.fit(xtrain2 , ytrain2)

Fitting 10 folds for each of 108 candidates, totalling 1080 fits
[CV] model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None, model__min_samples_split=2, model__n_estimators=10, preprocess__num__imputer__strategy=mean 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None, model__min_samples_split=2, model__n_estimators=10, preprocess__num__imputer__strategy=mean, total=   1.1s
[CV] model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None, model__min_samples_split=2, model__n_estimators=10, preprocess__num__imputer__strategy=mean 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV]  model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None, model__min_samples_split=2, model__n_estimators=10, preprocess__num__imputer__strategy=mean, total=   1.1s
[CV] model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None, model__min_samples_split=2, model__n_estimators=10, preprocess__num__imputer__strategy=mean 
[CV]  model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None, model__min_samples_split=2, model__n_estimators=10, preprocess__num__imputer__strategy=mean, total=   1.1s
[CV] model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None, model__min_samples_split=2, model__n_estimators=10, preprocess__num__imputer__strategy=mean 
[CV]  model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None, model__min_samples_split=2, model__n_estimators=10, preprocess__num__imputer__strategy=mean, total=   1.1s
[CV] model__max_depth=None, model__max_features=auto, model__max_leaf_nodes=None

...

[CV]  model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=4, model__n_estimators=15, preprocess__num__imputer__strategy=median, total=   0.1s
[CV] model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=10, preprocess__num__imputer__strategy=mean 
[CV]  model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=10, preprocess__num__imputer__strategy=mean, total=   0.1s
[CV] model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=10, preprocess__num__imputer__strategy=mean 
[CV]  model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=10, preprocess__num__imputer__strategy=mean, total=   0.1s
[CV] model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, mode

[CV]  model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=15, preprocess__num__imputer__strategy=mean, total=   0.1s
[CV] model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=15, preprocess__num__imputer__strategy=mean 
[CV]  model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=15, preprocess__num__imputer__strategy=mean, total=   0.1s
[CV] model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=15, preprocess__num__imputer__strategy=mean 
[CV]  model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model__n_estimators=15, preprocess__num__imputer__strategy=mean, total=   0.1s
[CV] model__max_depth=4, model__max_features=auto, model__max_leaf_nodes=5, model__min_samples_split=6, model_

[Parallel(n_jobs=1)]: Done 1080 out of 1080 | elapsed:  3.8min finished


Wall time: 3min 49s


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocess',
                                        ColumnTransformer(transformers=[('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(fill_value='missing',
                                                                                                        strategy='constant')),
                                                                                         ('encoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['pet_id',
                                                                          'issue_date',
                                                                          'listing_dat

In [62]:
gs_model_2.score(xtest2 , ytest2)

0.8757632067958587

In [63]:
from joblib import dump
dump(gs_model_2 , filename="pet_classifier_gridsearch.joblib")

['pet_classifier_gridsearch.joblib']

In [64]:
pet_predict1 = gs_model_2.predict(xtest2)
pet_predict1 = pd.Series(pet_predict1)

In [65]:
pet_predict1.head()

0    2
1    2
2    4
3    2
4    1
dtype: int64

In [66]:
ytest2.head()

7761     2
13403    2
10581    4
15173    2
11605    2
Name: pet_category, dtype: int64

In [67]:
pet_predict1.value_counts()

2    2371
1    1269
4     122
0       5
dtype: int64

In [68]:
ytest2.value_counts()

2    2212
1    1352
4     183
0      20
Name: pet_category, dtype: int64

In [70]:
data_test = pd.read_csv("C:/Users/dell/Desktop/ml/pet detectoin/Dataset/test.csv" , low_memory=False)

In [71]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8072 entries, 0 to 8071
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   pet_id        8072 non-null   object 
 1   issue_date    8072 non-null   object 
 2   listing_date  8072 non-null   object 
 3   condition     7453 non-null   float64
 4   color_type    8072 non-null   object 
 5   length(m)     8072 non-null   float64
 6   height(cm)    8072 non-null   float64
 7   X1            8072 non-null   int64  
 8   X2            8072 non-null   int64  
dtypes: float64(3), int64(2), object(4)
memory usage: 567.7+ KB


In [72]:
submit_file = pd.DataFrame()
submit_file["pet_id"] = data_test["pet_id"]

In [73]:
test_breed_predictor = pd.Series(gs_model.predict(data_test))
test_pet_predictor = pd.Series(gs_model_2.predict(data_test))

In [74]:
submit_file["breed_category"] = test_breed_predictor
submit_file["pet_category"] = test_pet_predictor

In [75]:
submit_file.shape

(8072, 3)

In [76]:
data_test.shape

(8072, 9)

In [77]:
submit_file.head()

,pet_id,breed_category,pet_category
0,ANSL_75005,1.0,2
1,ANSL_76663,0.0,1
2,ANSL_58259,0.0,2
3,ANSL_67171,0.0,1
4,ANSL_72871,0.0,2


In [78]:
data_test.head()

,pet_id,issue_date,listing_date,condition,color_type,length(m),height(cm),X1,X2
0,ANSL_75005,2005-08-17 00:00:00,2017-09-07 15:35:00,0.0,Black,0.87,42.73,0,7
1,ANSL_76663,2018-11-15 00:00:00,2019-05-08 17:24:00,1.0,Orange Tabby,0.06,6.71,0,1
2,ANSL_58259,2012-10-11 00:00:00,2018-04-02 16:51:00,1.0,Black,0.24,41.21,0,7
3,ANSL_67171,2015-02-13 00:00:00,2018-04-06 07:25:00,1.0,Black,0.29,8.46,7,1
4,ANSL_72871,2017-01-18 00:00:00,2018-04-26 13:42:00,1.0,Brown,0.71,30.92,0,7


In [79]:
submit_file.to_csv("Final_submit1.csv" , index=False)